In [5]:
from __future__ import division      #除数可以显示为float

from six import StringIO    #使用聚宽readfile函数
import numpy as np
import pandas as pd
import time                 #使用time stamp
import datetime             
import matplotlib.pyplot as plt
import math



# 定义分组函数，对输入的数列按照n组进行划分
def get_group(sample_data):
    # 返回结果的数组
    ret_group = []
    if len(sample_data)!=0:
        # 确定最大值、最小值
        # 同时gap适当扩大，避免出现n+1的分组
        d_mean = np.mean(sample_data)
        d_std = np.std(sample_data)
        ret_group = [math.floor(vals) if abs(math.floor(vals)) <=3 else 4*abs(math.floor(vals))/math.floor(vals) for vals in (np.array(sample_data) - d_mean)/(0.5*d_std)]
    else:
        ret_group =  []
    return ret_group

循环读取文件，生成归一化文件后保存

In [6]:
for file_i in range(1,2,1):
    
    Data_Total = pd.DataFrame()
    
    file_name = "raw_data_test.csv"
    
    Data_Total = pd.read_csv(file_name)
    
    Stock_list = set(list(Data_Total['code']))
    
   #############################################  1、 增加一个前一天的值      ############################################################
    #增加MACD_macd_pre
    Data_Total.loc[:,"MACD_macd_pre"] = None
    for stock_name in Stock_list:
        temp = []
        temp = list(Data_Total.loc[Data_Total['code']==stock_name,"MACD_macd"])[1:]
        temp.append(0)
        Data_Total.loc[Data_Total['code']==stock_name,"MACD_macd_pre"] \
        = np.array(temp)

    #增加:EMA_gap_pre
    Data_Total.loc[:,"EMA_gap_pre"] = None
    for stock_name in Stock_list:
        temp = []
        temp = list(Data_Total.loc[Data_Total['code']==stock_name,"EMA_gap"])[1:]
        temp.append(0)
        Data_Total.loc[Data_Total['code']==stock_name,"EMA_gap_pre"] \
        = np.array(temp)

    #增加:KDJ_J_pre
    Data_Total.loc[:,"KDJ_J_pre"] = None
    for stock_name in Stock_list:
        temp = []
        temp = list(Data_Total.loc[Data_Total['code']==stock_name,"KDJ_J"])[1:]
        temp.append(0)
        Data_Total.loc[Data_Total['code']==stock_name,"KDJ_J_pre"] \
        = np.array(temp)


    #增加:MOM_pre
    Data_Total.loc[:,"MOM_pre"] = None
    for stock_name in Stock_list:
        temp = []
        temp = list(Data_Total.loc[Data_Total['code']==stock_name,"MOM_12"])[1:]
        temp.append(0)
        Data_Total.loc[Data_Total['code']==stock_name,"MOM_pre"] \
        = np.array(temp)

    #增加:MOM_gap_pre
    Data_Total.loc[:,"MOM_gap_pre"] = None
    for stock_name in Stock_list:
        temp = []
        temp = list(Data_Total.loc[Data_Total['code']==stock_name,"MOM_gap"])[1:]
        temp.append(0)
        Data_Total.loc[Data_Total['code']==stock_name,"MOM_gap_pre"] \
        = np.array(temp)

    #增加:OBV_pre
    Data_Total.loc[:,"OBV_pre"] = None
    for stock_name in Stock_list:
        temp = []
        temp = list(Data_Total.loc[Data_Total['code']==stock_name,"OBV"])[1:]
        temp.append(0)
        Data_Total.loc[Data_Total['code']==stock_name,"OBV_pre"] \
        = np.array(temp)

    #增加:Long_Short_Rate_OBV_pre
    Data_Total.loc[:,"Long_Short_Rate_OBV_pre"] = None
    for stock_name in Stock_list:
        temp = []
        temp = list(Data_Total.loc[Data_Total['code']==stock_name,"Long_Short_Rate_OBV"])[1:]
        temp.append(0)
        Data_Total.loc[Data_Total['code']==stock_name,"Long_Short_Rate_OBV_pre"] \
        = np.array(temp)
        
   #############################################  2、 删除无效数据，更改数据类型      ############################################################

    # 删除前30，后10的数据
    for stock_name in Stock_list:
        Data_Total = Data_Total.drop(Data_Total[Data_Total['code'] == stock_name].iloc[:30].index, axis=0)
        Data_Total = Data_Total.drop(Data_Total[Data_Total['code'] == stock_name].iloc[-10:].index, axis=0)


    # 统一操作，删除价格为0的数据
    Data_Total = Data_Total.drop(Data_Total[Data_Total['price'] == 0].index, axis=0)

#     # 对win_rate进行操作，按照“win”为+1，“even”为0，“lose”为-1来进行
#     Data_Total.loc[Data_Total['win_rate'] == "win","win_rate"] = float(1)
#     Data_Total.loc[Data_Total['win_rate'] == "lose","win_rate"] = float(-1)
#     Data_Total.loc[Data_Total['win_rate'] == "even","win_rate"] = float(0)

    # 更改类型为float
    Data_Total[['win_rate']] = Data_Total[['win_rate']].astype(float)
    # 更改平均为-1
    Data_Total[Data_Total['win_rate'] == 0] = -1
    
    
   #############################################  3、 归一化数据      ############################################################
    Data_Normalize = pd.DataFrame()

    # 赋值列：code、date，avg_price，win_rate
    Data_Normalize = Data_Total[['code','date','price','win_rate']]

    # 赋值列：MACD_macd
    Data_Normalize.loc[:,"MACD_macd"] = None

    for stock_name in Stock_list:
        Data_Normalize.loc[Data_Normalize['code']==stock_name,"MACD_macd"] \
        = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["MACD_macd"])))

    # 赋值列：EMA_gap
    Data_Normalize.loc[:,"EMA_gap"] = None

    for stock_name in Stock_list:
        Data_Normalize.loc[Data_Normalize['code']==stock_name,"EMA_gap"] \
        = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["EMA_gap"])))


    # 赋值列：KDJ_J
    Data_Normalize.loc[:,"KDJ_J"] = None

    for stock_name in Stock_list:
        Data_Normalize.loc[Data_Normalize['code']==stock_name,"KDJ_J"] \
        = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["KDJ_J"])))

    # 赋值列：RSI
    Data_Normalize.loc[:,"RSI"] = None

    for stock_name in Stock_list:
        Data_Normalize.loc[Data_Normalize['code']==stock_name,"RSI"] \
        = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["RSI"])))

    # 赋值列：MOM
    Data_Normalize.loc[:,"MOM_12"] = None

    for stock_name in Stock_list:
        Data_Normalize.loc[Data_Normalize['code']==stock_name,"MOM_12"] \
        = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["MOM_12"])))


    # 赋值列：MOM_gap
    Data_Normalize.loc[:,"MOM_gap"] = None

    for stock_name in Stock_list:
        Data_Normalize.loc[Data_Normalize['code']==stock_name,"MOM_gap"] \
        = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["MOM_gap"])))


    # 赋值列：OBV
    Data_Normalize.loc[:,"OBV"] = None

    for stock_name in Stock_list:
        Data_Normalize.loc[Data_Normalize['code']==stock_name,"OBV"] \
        = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["OBV"])))



    # 赋值列：Long_Short_Rate_OBV
    Data_Normalize.loc[:,"Long_Short_Rate_OBV"] = None

    for stock_name in Stock_list:
        Data_Normalize.loc[Data_Normalize['code']==stock_name,"Long_Short_Rate_OBV"] \
        = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["Long_Short_Rate_OBV"])))


    # KDJ_J_pre  MOM_pre  MOM_gap_pre     OBV_pre Long_Short_Rate_OBV_pre  

    # 赋值列：KDJ_J_pre
    Data_Normalize.loc[:,"KDJ_J_pre"] = None

    for stock_name in Stock_list:
        Data_Normalize.loc[Data_Normalize['code']==stock_name,"KDJ_J_pre"] \
        = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["KDJ_J_pre"])))

    # 赋值列：MOM_pre
    Data_Normalize.loc[:,"MOM_pre"] = None

    for stock_name in Stock_list:
        Data_Normalize.loc[Data_Normalize['code']==stock_name,"MOM_pre"] \
        = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["MOM_pre"])))

    # 赋值列：MOM_gap_pre
    Data_Normalize.loc[:,"MOM_gap_pre"] = None

    for stock_name in Stock_list:
        Data_Normalize.loc[Data_Normalize['code']==stock_name,"MOM_gap_pre"] \
        = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["MOM_gap_pre"])))

    # 赋值列：OBV_pre
    Data_Normalize.loc[:,"OBV_pre"] = None

    for stock_name in Stock_list:
        Data_Normalize.loc[Data_Normalize['code']==stock_name,"OBV_pre"] \
        = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["OBV_pre"])))

    # 赋值列：Long_Short_Rate_OBV_pre
    Data_Normalize.loc[:,"Long_Short_Rate_OBV_pre"] = None

    for stock_name in Stock_list:
        Data_Normalize.loc[Data_Normalize['code']==stock_name,"Long_Short_Rate_OBV_pre"] \
        = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["Long_Short_Rate_OBV_pre"])))


    # MACD_macd_pre  EMA_gap_pre
    # 赋值列：MACD_macd_pre
    Data_Normalize.loc[:,"MACD_macd_pre"] = None

    for stock_name in Stock_list:
        Data_Normalize.loc[Data_Normalize['code']==stock_name,"MACD_macd_pre"] \
        = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["MACD_macd_pre"])))

    # 赋值列：EMA_gap_pre
    Data_Normalize.loc[:,"EMA_gap_pre"] = None

    for stock_name in Stock_list:
        Data_Normalize.loc[Data_Normalize['code']==stock_name,"EMA_gap_pre"] \
        = np.array(get_group(list(Data_Total[Data_Total['code']==stock_name]["EMA_gap_pre"])))

        
   #############################################  4、生成文件     ############################################################
    
    file_name_normalize = 'Data_Washed.csv'
    Data_Normalize.to_csv(file_name_normalize)